In [2]:
import argparse
import time
import cv2
from cv2 import dnn_superres
import numpy as np
import os
# from utils import crop

In [70]:
"""
new clip
"""
def clip_one_picture(src,cols,rows):
    img=cv2.imread(src,1)
    sum_rows=img.shape[0]   #height
    sum_cols=img.shape[1]   #width
    tmp=src.split('super_resolution')[0]+"super_resolution/{0}_croped/".format(src.split('/')[-1].split('.')[0])
    # TODO: check temp path
    if not os.path.exists(tmp):
        os.makedirs(tmp)
    print("裁剪所得{0}列图片，{1}行图片.".format(int(sum_cols/cols),int(sum_rows/rows)))
    num_cols = int(sum_cols/cols)
    num_rows = int(sum_rows/rows)

    for i in range(num_cols):
        for j in range(num_rows):
            cv2.imwrite(tmp+'temp_'+str(j)+'_'+str(i)+'.'+src.split('.')[-1], img[j*rows:(j+1)*rows,i*cols:(i+1)*cols,:])
    return tmp, num_cols, num_rows

"""
输入：图片路径(path+filename)，裁剪获得小图片的列数、行数（也即宽、高）

def clip_one_picture(path,filename,cols,rows):
    img=cv2.imread(path+filename,-1)##读取彩色图像，图像的透明度(alpha通道)被忽略，默认参数;灰度图像;读取原始图像，包括alpha通道;可以用1，0，-1来表示
    sum_rows=img.shape[0]   #高度
    sum_cols=img.shape[1]    #宽度
    save_path=path+"/{0}_croped_{1}x{2}/".format(filename,cols,rows)  #保存的路径
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    print("裁剪所得{0}列图片，{1}行图片.".format(int(sum_cols/cols),int(sum_rows/rows)))
    num_cols = int(sum_cols/cols)
    num_rows = int(sum_rows/rows)

    for i in range(num_cols):
        for j in range(num_rows):
            cv2.imwrite(save_path+os.path.splitext(filename)[0]+'_'+str(j)+'_'+str(i)+os.path.splitext(filename)[1],img[j*rows:(j+1)*rows,i*cols:(i+1)*cols,:])
    # print("裁剪完成，得到{0}张图片.".format(int(sum_cols/cols)*int(sum_rows/rows)))
    print("裁剪所得图片的存放地址为：{0}".format(save_path))
    return save_path, num_cols, num_rows
"""

"""调用裁剪函数示例"""
# file_path = "./examples/big/" 
# img_path = "child.png"

# cols=600        #小图片的宽度（列数）
# rows=600     #小图片的高度（行数）
# clip_one_picture(path=file_path,filename=img_path,cols=cols,rows=rows)


"""

输入：图片路径(path+filename)，裁剪所的图片的列的数量、行的数量
输出：无
"""

def merge_picture(merge_path,num_of_cols,num_of_rows):
    filename=file_name(merge_path,".png")
#     print(merge_path, filename)
    shape=cv2.imread(filename[0],-1).shape    #三通道的影像需把-1改成1
    cols=shape[1]
    rows=shape[0]
    channels=shape[2]
    dst=np.zeros((rows*num_of_rows,cols*num_of_cols,channels),np.uint8)
    for i in range(len(filename)):
        img=cv2.imread(filename[i],-1)
        cols_th=int(filename[i].split("_")[-1].split('.')[0])
        rows_th=int(filename[i].split("_")[-2])
        roi=img[0:rows,0:cols,:]
        dst[rows_th*rows:(rows_th+1)*rows,cols_th*cols:(cols_th+1)*cols,:]=roi
        # TODO: remove temp files
    # cv2.imwrite(merge_path+"merge.png",dst, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])
    return dst

"""遍历文件夹下某格式图片"""
def file_name(root_path,picturetype):
    filename=[]
    for root,dirs,files in os.walk(root_path):
        for file in files:
            if os.path.splitext(file)[1]==picturetype:
                filename.append(os.path.join(root,file))
    return filename



In [71]:
class opencvsr:
    def __init__(self, mpath, scale):
        self.model = mpath
        self.sr = dnn_superres.DnnSuperResImpl_create()
        
    def upscale_normal_pic(self, src, dst):
        orig = cv2.imread(src, 1)
        self.sr.readModel(self.model)
        self.sr.setModel(os.path.split(model)[-1].split('_')[0].lower(), self.scale)
        upscaled = self.sr.upsample(orig)
        cv2.imwrite(dst, upscaled, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])
        
    def upscale_large_pic(self, src, dst):
        orig = cv2.imread(src, 1)
        self.sr.readModel(self.model)
        self.sr.setModel(os.path.split(model)[-1].split('_')[0].lower(), self.scale)
        cpath, num_cols, num_rows = clip_one_picture(src=src,cols=60,rows=60)
        for i, filename in enumerate(os.listdir(cpath)):
            img_path = cpath + filename
            croped = cv2.imread(img_path, 1)
            upscaled = self.sr.upsample(croped)
            cv2.imwrite(img_path, upscaled, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])

        merged = merge_picture(cpath,num_of_cols=num_cols,num_of_rows=num_rows)
        cv2.imwrite(dst, merged, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])

In [72]:
mpath = '/Users/panyining/Desktop/sigrid/Code/basic-computer-version/super_resolution/models/ESPCN_x4.pb'
src = '/Users/panyining/Desktop/sigrid/Code/basic-computer-version/super_resolution/examples/examples/zebra.png'
dst = '/Users/panyining/Desktop/sigrid/Code/basic-computer-version/super_resolution/examples/output/' + src.split('/')[-1].split('.png')[0]+'.png'
a = opencvsr(mpath=mpath, scale=4)
a.upscale_large_pic(src=src, dst=dst)

裁剪所得6列图片，4行图片.


In [ ]:
import argparse
import time
import cv2
from cv2 import dnn_superres
import os
from utils import crop


def super_res_image(img_path, out_path, model_path):
    modelName = model_path.split("/")[-1].split("_")[0].lower()
    modelScale = model_path.split("_x")[-1]
    modelScale = int(modelScale[:modelScale.find(".")])
    sr = dnn_superres.DnnSuperResImpl_create()
    sr.readModel(model_path)
    sr.setModel(modelName, modelScale)
    _, filename = os.path.split(img_path)
    cols=600
    rows=600

    if os.path.splitext(filename)[-1] not in (".png", ".jpg", ".jpeg", ".bmp"):
        print("can't open file: {filename}")
    else:
        orig = cv2.imread(img_path, 1)
        if orig.shape[0]*orig.shape[1] <= cols*rows:
            upscaled = sr.upsample(orig)
            out_filename = filename.split(".")[0] + "_" + str(modelName) + "_x" + str(modelScale) + ".png"
            cv2.imwrite(out_path + "/" + str(out_filename), upscaled, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])
        else:
            # crop, sr then merge
            crop_path, num_cols, num_rows = crop.clip_one_picture(path=os.path.split(img_path)[0],filename=os.path.split(img_path)[-1],cols=cols,rows=rows)
            for i, filename in enumerate(os.listdir(crop_path)):
                img_path = crop_path + filename
                croped = cv2.imread(img_path, 1)
                upscaled = sr.upsample(croped)
                cv2.imwrite(img_path, upscaled, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])
                
            merge_upscaled = crop.merge_picture(crop_path,sum_cols=modelScale*orig.shape[0],sum_rows=modelScale*orig.shape[1],cols=modelScale*cols,rows=modelScale*rows)
            out_filename = filename.split(".")[0] + "_" + str(modelName) + "_x" + str(modelScale) + ".png"
            cv2.imwrite(out_path + "/" + str(out_filename), merge_upscaled, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])

def super_res(file_path, out_path, model_path):

    # modelName = model_path.split("/")[-1].split("_")[0].lower()
    # modelScale = model_path.split("_x")[-1]
    # modelScale = int(modelScale[:modelScale.find(".")])
    # sr = dnn_superres.DnnSuperResImpl_create()
    # sr.readModel(model_path)
    # sr.setModel(modelName, modelScale)

    # check output and input(file or dir) path
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    if os.path.isfile(file_path):
        super_res_image(img_path=file_path, out_path=out_path, model_path=model_path)
    elif os.path.isdir(file_path):
        for filename in os.listdir(file_path):
            img_path = file_path + "/" + filename
            super_res_image(img_path=img_path, out_path=out_path, model_path=model_path)
    else:
        print(f"cannot read path: {file_path}, please try again.")

    return


def main():
    model_path = "./models/FSRCNN_x3.pb"
    file_path = "./examples/" 
    img_path = file_path + "pexels-any-lane-5727731.png"
    out_path = file_path + "result"
    super_res(file_path, out_path, model_path)
    print("success!!!")


if __name__ == "__main__":
    main()